In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [1]:
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, month, year,monotonically_increasing_id,row_number
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType,TimestampNTZType,StringType
from pyspark.sql.window import Window
from pyspark.conf import SparkConf
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime, date
import os
from dotenv import load_dotenv, dotenv_values
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
from binance.exceptions import BinanceRequestException, BinanceAPIException
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import time
import logging
import psycopg2
load_dotenv()

True

In [2]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [3]:
#conf = SparkConf()  # create the configuration
#conf.set("spark.jars", "/Users/hamza/.local/share/virtualenvs/DE-Crypto-aeKJmBel/lib/python3.9/site-packages/pyspark/jars/Users/hamza/.local/share/virtualenvs/DE-Crypto-aeKJmBel/lib/python3.9/site-packages/pyspark/jars/postgresql-42.7.5.jar")  # set the spark.jars


In [4]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .config("spark.jars", "/Users/hamza/Desktop/projects/postgresql-42.7.5.jar") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

25/04/20 20:41:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/20 20:41:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/20 20:41:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/20 20:41:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
#read data for 1 crypto
#schema check/validation need to do this, create spark df using schema struct
#implement transformations
#add unique id for crypto,price,time
#split into 3 tables
#upload to postgres db

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:1: E265 block comment should start with '# '
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '


In [6]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #set to debug to capture all levels
if logger.hasHandlers():
    logger.handlers.clear()
logger.propagate = False


2:31: E261 at least two spaces before inline comment
2:32: E262 inline comment should start with '# '
6:1: W391 blank line at end of file


In [7]:
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [8]:
cryptos = ['BTCUSDT','ETHUSDT','LTCUSDT','BNBUSDT','DOGEUSDT']
client_minio = Minio(
        "localhost:9000",  # Make sure you're using port 9000 for the S3 API
        #minio_url,
        access_key = MINIO_USER,
        secret_key = MINIO_PASSWORD,
        secure=False  # Disable SSL if you're not using SSL certificates
        )

1:21: E231 missing whitespace after ','
1:31: E231 missing whitespace after ','
1:41: E231 missing whitespace after ','
1:51: E231 missing whitespace after ','
4:9: E265 block comment should start with '# '
5:19: E251 unexpected spaces around keyword / parameter equals
5:21: E251 unexpected spaces around keyword / parameter equals
6:19: E251 unexpected spaces around keyword / parameter equals
6:21: E251 unexpected spaces around keyword / parameter equals


In [9]:
'''client_minio.fget_object(bucket_name = "binancedata", 
                         object_name ="BNBUSDT/Monthly/year=2019/month=1/part-00000-c9e58e49-1c79-4e98-b1a7-f0611e7acc19.c000.snappy.parquet", 
                         file_path ="./downloaded.parquet"
                        )'''
#binancedata/BNBUSDT/Monthly/year=2019/month=1
#binancedata/BNBUSDT/Monthly/year=2019/month=1/part-00000-c9e58e49-1c79-4e98-b1a7-f0611e7acc19.c000.snappy.parquet

'client_minio.fget_object(bucket_name = "binancedata", \n                         object_name ="BNBUSDT/Monthly/year=2019/month=1/part-00000-c9e58e49-1c79-4e98-b1a7-f0611e7acc19.c000.snappy.parquet", \n                         file_path ="./downloaded.parquet"\n                        )'

1:57: W291 trailing whitespace
2:80: E501 line too long (142 > 79 characters)
2:143: W291 trailing whitespace
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '
6:80: E501 line too long (114 > 79 characters)


In [10]:
objects = client_minio.list_objects("binancedata", prefix="BNBUSDT", recursive=True)
filenames = [obj.object_name for obj in objects]
filenames = [f for f in filenames if "_SUCCESS" not in f]

1:80: E501 line too long (84 > 79 characters)


In [11]:
'''client_minio.fget_object(bucket_name = "binancedata", 
                         object_name =filenames[0], 
                         file_path =local_file_path
                        )'''

'client_minio.fget_object(bucket_name = "binancedata", \n                         object_name =filenames[0], \n                         file_path =local_file_path\n                        )'

1:57: W291 trailing whitespace
2:52: W291 trailing whitespace


In [12]:
'''schema = StructType([\
                    StructField(name = 'Date',dataType = TimestampNTZType(),nullable = False), \
                    StructField(name= 'Open Price',dataType = LongType(),nullable =False), \
                    StructField(name= 'Close Price',dataType = LongType(),nullable =False), \
                    StructField(name= 'Volume',dataType = LongType(),nullable = False)\
                                ])
                    '''

"schema = StructType([                    StructField(name = 'Date',dataType = TimestampNTZType(),nullable = False),                     StructField(name= 'Open Price',dataType = LongType(),nullable =False),                     StructField(name= 'Close Price',dataType = LongType(),nullable =False),                     StructField(name= 'Volume',dataType = LongType(),nullable = False)                                ])\n                    "

2:80: E501 line too long (96 > 79 characters)
3:80: E501 line too long (92 > 79 characters)
4:80: E501 line too long (93 > 79 characters)
5:80: E501 line too long (87 > 79 characters)


In [13]:
df_bnb = spark.read.parquet("s3a://binancedata/BNBUSDT/Monthly/year=2019/month=3/part-00000-4770a452-7828-40a3-b901-c42eda5f9c00.c000.snappy.parquet")

25/04/20 14:17:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
1:80: E501 line too long (150 > 79 characters)


In [14]:
df_bnb.show(10)

+-------------------+-----------+-----------+------------------+
|           datetime| Open Price|Close Price|            Volume|
+-------------------+-----------+-----------+------------------+
|2019-03-01 00:00:00|10.27770000|17.53760000|115379333.44000000|
+-------------------+-----------+-----------+------------------+



In [15]:
for file in filenames:
    df = spark.read.parquet(f"s3a://binancedata/{file}")
    df_bnb = df_bnb.union(df)
                            
    
    

4:1: W293 blank line contains whitespace
5:1: W293 blank line contains whitespace
6:1: W293 blank line contains whitespace


In [16]:
#do transformations when its in spark df
#push to postgres

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '


In [17]:
df_bnb.show(5)

+-------------------+-----------+-----------+------------------+
|           datetime| Open Price|Close Price|            Volume|
+-------------------+-----------+-----------+------------------+
|2019-03-01 00:00:00|10.27770000|17.53760000|115379333.44000000|
|2019-01-01 00:00:00| 6.11390000| 6.22000000| 72126339.77000000|
|2019-10-01 01:00:00|15.84930000|19.90990000| 53481014.91000000|
|2019-11-01 00:00:00|19.90870000|15.71180000| 46441747.10000000|
|2019-12-01 00:00:00|15.70300000|13.71610000| 38189582.55000000|
+-------------------+-----------+-----------+------------------+
only showing top 5 rows



In [18]:
def data_cleaning():
    pass
    #drop duplicates
    #convert data types
    #drop null values

    

3:5: E265 block comment should start with '# '
4:5: E265 block comment should start with '# '
5:5: E265 block comment should start with '# '
7:1: W293 blank line contains whitespace


In [19]:
def df_to_table():
    pass\
    #read from postgres
    #convert table to df and join with dataframe
    #check if id
    #convert dataframe to sql tables

3:5: E265 block comment should start with '# '


In [20]:
df_bnb

DataFrame[datetime: timestamp, Open Price: string, Close Price: string, Volume: string]

25/04/20 14:17:27 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [24]:
conn = psycopg2.connect(
    database="crypto",  # Switch to the 'crypto' database
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432",
)
conn.autocommit = True
cursor = conn.cursor()

In [5]:
crypto_dict = [{"name":"Bitcoin","ticker":"BTC"},
               {"name":"Ethereum","ticker":"ETH"},
               {"name":"Litecoin","ticker":"LTC"},
               {"name":"Dogecoin","ticker":"DOGE"},
               {"name":"BNB","ticker":"BNB"}
              ]
crypto_df = spark.createDataFrame(crypto_dict)
#crypto_df = crypto_df.withColumn('id',monotonically_increasing_id())

In [6]:
windowSpec = Window.orderBy("name")

# Add a sequential ID column starting from 10
crypto_df = crypto_df.withColumn("id", row_number().over(windowSpec) + 9)

In [7]:
crypto_df.head(5)

25/04/20 20:41:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/20 20:41:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/20 20:41:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
                                                                                

[Row(name='BNB', ticker='BNB', id=10),
 Row(name='Bitcoin', ticker='BTC', id=11),
 Row(name='Dogecoin', ticker='DOGE', id=12),
 Row(name='Ethereum', ticker='ETH', id=13),
 Row(name='Litecoin', ticker='LTC', id=14)]

In [8]:
crypto_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/crypto") \
    .option("dbtable", "crypto") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append")\
    .save()

25/04/20 20:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/20 20:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/20 20:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/20 20:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/20 20:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [7]:
cursor.execute(sql_tables)
print("Tables Crypto, Time and Price have been created successfully !!")

# Closing the connection
conn.close()

NameError: name 'cursor' is not defined

In [11]:
print(crypto_df.printSchema())

In [24]:
crypto_df = crypto_df.drop("id")


2:1: W391 blank line at end of file
